In [1]:
import itertools
import h5py
import numpy as np
import pandas
from einops import rearrange
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transformers
import matplotlib.pyplot as plt
from torch.autograd import Variable


In [2]:
class Encoder(nn.Module): 
    def __init__(self, bottleneck_size):
        super(Encoder,self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(5,3), stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels=64, kernel_size=(5,3), stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=128, kernel_size=3,stride=2, padding=0),
            nn.ReLU()
        )

        self.dense_layers = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(128 * 30* 12 , 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512,bottleneck_size),
           )
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.dense_layers(x)
        return x


In [3]:
class Decoder(nn.Module):
    def __init__(self, bottleneck_size ):
        super(Decoder,self).__init__()
        self.conv_layers = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=64,out_channels=32, kernel_size=(5,3), stride=2, padding=1, output_padding=(1,0)),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=32,out_channels=1, kernel_size=(5,3),stride=2, padding=1, output_padding=1),
            nn.ReLU()
        )

        self.unflatten = nn.Unflatten(dim=1, unflattened_size=(128,30,12))

        self.dense_layers = nn.Sequential(

            nn.Linear(bottleneck_size,1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 128 * 30 * 12),
            nn.ReLU(),
            nn.Dropout(0.2),
           )

    def forward(self, x):
        x = self.dense_layers(x)
        x  = self.unflatten(x)
        x = self.conv_layers(x)
        x = x[:, :, 4:254, 2:102]

        return x


In [4]:
class AE(nn.Module):

    def __init__(self, encoder, decoder, bottleneck_size=1024):
        super(AE,self).__init__()
        self.encoder = encoder(bottleneck_size)
        self.decoder = decoder(bottleneck_size)
    def forward(self,x ):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [5]:
ae_pretrain = AE(Encoder, Decoder)

In [6]:
ae_pretrain.load_state_dict(torch.load('./logging/model.pth'))

<All keys matched successfully>

In [7]:
#ae_pretrain.eval()

In [8]:
from utils.data_loading import CombustionSystemDataset

PATH = 'combustion_img_13.mat'

train_data = CombustionSystemDataset(PATH,'train_set_x','train_set_y')
test_data = CombustionSystemDataset(PATH,'test_set_x','test_set_y')

In [9]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=6, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=6, shuffle=False)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [11]:
ae_pretrain.to(device)

AE(
  (encoder): Encoder(
    (conv_layers): Sequential(
      (0): Conv2d(1, 32, kernel_size=(5, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(5, 3), stride=(2, 2), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
      (5): ReLU()
    )
    (dense_layers): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=46080, out_features=512, bias=True)
      (2): ReLU()
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=512, out_features=1024, bias=True)
    )
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (decoder): Decoder(
    (conv_layers): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), output_padding=(1, 1))
      (1): ReLU()
      (2): ConvTranspose2d(64, 32, kernel_size=(5, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 0))
      (3): ReLU()
      (4): ConvTranspose2d(32, 1, kernel_size=(5, 3), s

In [12]:
#train_codes_stashed = torch.empty(1,1024).to(device)
#train_targets_stashed = torch.empty(1,1).to(device)
batch_ct=0   #Diagnositic
for train_images, train_targets in train_loader:
    batch_ct=batch_ct+1
    train_images, train_targets = train_images.to(device), train_targets.to(device)
    train_codes = ae_pretrain.encoder(train_images)
    
    #train_images=train_images.to('cpu')
    #train_codes=train_codes.to('cpu')
    #train_targets=train_targets.to('cpu')
    
    #train_codes_stashed = torch.cat((train_codes_stashed, train_codes),dim=0)
    #train_targets_stashed = torch.cat((train_targets_stashed, train_targets),dim=0)
    
    #Diagnositic
  
    df_code = pandas.DataFrame(train_codes.to('cpu').detach().numpy())
    df_class=pandas.DataFrame(train_targets.to('cpu').detach().numpy())
    df_code.to_csv('train_embbeds.csv', mode='a', index=False, header=False)
    df_class.to_csv('train_labels.csv', mode='a', index=False, header=False)

    if batch_ct % 500 == 0:
        print("Batch: ",batch_ct)

Batch:  500
Batch:  1000
Batch:  1500
Batch:  2000
Batch:  2500
Batch:  3000
Batch:  3500
Batch:  4000
Batch:  4500
Batch:  5000
Batch:  5500
Batch:  6000
Batch:  6500
Batch:  7000
Batch:  7500
Batch:  8000
Batch:  8500
Batch:  9000


In [16]:
batch_ct=0   #Diagnositic
for test_images, test_targets in test_loader:
    
    test_images, test_targets = test_images.to(device), test_targets.to(device)
    test_codes = ae_pretrain.encoder(test_images)
  
    df_code = pandas.DataFrame(test_codes.to('cpu').detach().numpy())
    df_class=pandas.DataFrame(test_targets.to('cpu').detach().numpy())
    df_code.to_csv('test_embbeds.csv', mode='a', index=False, header=False)
    df_class.to_csv('test_labels.csv', mode='a', index=False, header=False)

    batch_ct=batch_ct+1
    if batch_ct % 500 == 0:
        print("Batch: ",batch_ct)

Batch:  500
Batch:  1000
Batch:  1500
Batch:  2000
Batch:  2500
Batch:  3000


In [18]:
valid_data = CombustionSystemDataset(PATH,'valid_set_x','valid_set_y')
valid_loader = torch.utils.data.DataLoader(dataset=valid_data, batch_size=6, shuffle=False)

In [19]:
batch_ct=0   #Diagnositic
for valid_images, valid_targets in valid_loader:
    
    valid_images, valid_targets = valid_images.to(device), valid_targets.to(device)
    valid_codes = ae_pretrain.encoder(valid_images)
  
    df_code = pandas.DataFrame(valid_codes.to('cpu').detach().numpy())
    df_class = pandas.DataFrame(valid_targets.to('cpu').detach().numpy())
    df_code.to_csv('valid_embbeds.csv', mode='a', index=False, header=False)
    df_class.to_csv('valid_labels.csv', mode='a', index=False, header=False)

    batch_ct=batch_ct+1
    if batch_ct % 500 == 0:
        print("Batch: ",batch_ct)

Batch:  500
Batch:  1000
Batch:  1500


In [20]:
print("The type of test_codes: ", type(test_codes))
print("The shape of test_codes: ", test_codes.size())
print("The shape of train_codes: ",train_codes.size())

The type of test_codes:  <class 'torch.Tensor'>
The shape of test_codes:  torch.Size([6, 1024])
The shape of train_codes:  torch.Size([6, 1024])


In [ ]:
def sliding_windows(dataX,dataY, seq_length):
    print("shape of data: ", dataX.size())
    print("shape of data labels: ", dataY.size())
    print("seq_length: ", seq_length)
    
    batch_len=len(dataX)
    
    x = torch.empty((1,4,1024))
    # y = torch.empty((1,1))
    y = torch.empty(1,1)
    
    for i in range(batch_len-seq_length-1):
        # _x = data.detach()[i:(i+seq_length)]
        # _y = data.detach()[i+seq_length]
        _x = dataX[i:(i+seq_length)]
        _y = dataY[i+seq_length]
        
        _xOut=rearrange(_x,'w h -> 1 w h')
        _yOut=rearrange(_y,'w -> 1 w')


        x = torch.cat((x, _xOut),dim=0)
        y = torch.cat((y, _yOut),dim=0)

    # return torch.tensor(x),torch.tensor(y)
    print("Size of x: ", x.size())
    print("Size of _x: ", _x.size())
    print("Size of y: ", y.size())
    print("Size of _y: ", _y.size())
    return x.clone().detach(), y.clone().detach()

In [ ]:
seq_length = 4
x_train, y_train = sliding_windows(train_codes_cpu, train_targets_cpu, seq_length)

print("The shape of x_train: ", x_train.size())
print("The length of x_train: ", len(x_train))
print("The shape of y_train: ", y_train.size())
print("The length of y_train: ", len(y_train))

In [ ]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers, device):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [ ]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

In [ ]:
trainX = Variable(x_train)
trainY = Variable(y_train)
    
print("Shape of trainX: ", trainX.size())
print("Shape of trainY: ", trainY.size())

In [ ]:
num_epochs = 1000
learning_rate = 0.001

input_size = 1024
hidden_size = 512
num_layers = 1

num_classes = 1

# locate our GPU device and initialize it for training
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

lstm = LSTM(num_classes, input_size, hidden_size, num_layers, device)
lstm.to(device)
print("Number of parameters: ", sum(p.numel() for p in lstm.parameters()))

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX.to(device))

    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY.to(device))


    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))